# Basic Models in TensorFlow
- Review
- Linear regression in TensorFlow
- Optimizers
- Logistic regression on MNIST
- Loss functions

## Steps
### Phase 1: Assemble our graph
1. Read in data
2. Create placeholders for inputs and labels
3. Create weight and bias
4. Build model to predict Y
5. Specify loss function
6. Create optimizer

### Phase 2: Train our model
1. initialize variables
2. run optimizer operation with data fed into placeholders for inputs and labels. 

### See your model in TensorBoard
1. `writer = tf.summary.FileWriter('folder', sess.graph)`
2. `tensorboard --logdir=folder`

### Plot the results with matplotlib
1. uncomment the plotting code at the end of your program
2. run it again. 

In [1]:
def huber_loss(labels, predictions, delta=1.0):
    residual = tf.abs(predictions - labels)
    condition = tf.less(residual, delta)
    small_res = 0.5 * tf.square(residual)
    large_res = delta * residual - 0.5*tf.square(delta)
    return tf.select(condition, small_res, large_res)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

1.3.0


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
MNIST = input_data.read_data_sets('MNIST_data/', one_hot=True)
print(MNIST.train.images.shape)
print(MNIST.train.labels.shape)
print(MNIST.validation.images.shape)
print(MNIST.validation.labels.shape)
print(MNIST.test.images.shape)
print(MNIST.test.labels.shape)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
(55000, 10)
(5000, 784)
(5000, 10)
(10000, 784)
(10000, 10)


In [3]:
batch_size = None
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [batch_size, 784], name='image')
    Y = tf.placeholder(tf.float32, [batch_size, 10], name='label')

W = tf.Variable(tf.zeros([784, 10]), name='W')
b = tf.Variable(tf.zeros([10]), name='b')

logits = tf.matmul(X, W) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
loss = tf.reduce_mean(entropy)

train_step = tf.train.GradientDescentOptimizer(learning_rate=.01).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('tmp/CS20SI3/', sess.graph)
    for i in range(500):
        batch_x, batch_y = MNIST.train.next_batch(500)
        sess.run(train_step, feed_dict = {X:batch_x, Y:batch_y})
        if i%500 == 0:
            print(sess.run(loss, feed_dict = {X:batch_x, Y:batch_y}) )
            print(sess.run(accuracy, feed_dict={X: batch_x, Y:batch_y}))


    print('Training set accuracy:')
    print(sess.run(accuracy, feed_dict={X:MNIST.train.images, Y:MNIST.train.labels}))
    print('Testing set accuracy:')
    print(sess.run(accuracy, feed_dict={X:MNIST.test.images, Y:MNIST.test.labels}))

2.29059
0.44
Training set accuracy:
0.84011
Testing set accuracy:
0.8503


![](Lecture-03-MNIST-softmax.png)